In [36]:
# importing sympy
import sympy as sp

In [37]:
n_element_nodes=8

# defining the symbols for the local and global coordinates
x, y, z = sp.symbols('x y z')
l_x, l_y, l_z = sp.symbols('l_x l_y l_z')

# defining the local coordinates as functions of the global coordinates
xi = x/l_x
eta = y/l_y
zeta = z/l_z

# defining the 8 shape functions for the 8 nodes as functions of xi, eta, zeta
N = sp.Matrix(n_element_nodes, 1, lambda i,_: sp.var("N_{i}".format(i=i)))

N[0] = (1-xi)*(1-eta)*(1-zeta)
N[1] = (xi)*(1-eta)*(1-zeta)
N[2] = (xi)*(eta)*(1-zeta)
N[3] = (1-xi)*(eta)*(1-zeta)
N[4] = (1-xi)*(1-eta)*zeta
N[5] = (xi)*(1-eta)*zeta
N[6] = (xi)*(eta)*zeta
N[7] = (1-xi)*(eta)*zeta

T = sp.Matrix(n_element_nodes, 1, lambda i,_: sp.var("T_{i}".format(i=i)))

In [38]:
# defining the derivatives of the shape functions with respect to the global coordinates
DN = sp.Matrix([[sp.diff(N[i], x), sp.diff(N[i], y), sp.diff(N[i], z)] for i in range(n_element_nodes)]).T
DN_cube = DN.subs({l_x: 1, l_y: 1, l_z: 1})
DN_cube

Matrix([
[-(1 - y)*(1 - z), (1 - y)*(1 - z), y*(1 - z),      -y*(1 - z),      -z*(1 - y), z*(1 - y), y*z,      -y*z],
[-(1 - x)*(1 - z),      -x*(1 - z), x*(1 - z), (1 - x)*(1 - z),      -z*(1 - x),      -x*z, x*z, z*(1 - x)],
[-(1 - x)*(1 - y),      -x*(1 - y),      -x*y,      -y*(1 - x), (1 - x)*(1 - y), x*(1 - y), x*y, y*(1 - x)]])

In [39]:
# defining the D as a diagonal matrix with d11, d22, d33 as the diagonal elements
d11, d22, d33 = sp.symbols('d11 d22 d33')
D = sp.diag(d11, d22, d33)
K = sp.Integral(DN.T * D * DN, (x, 0, l_x), (y, 0, l_y), (z, 0, l_z)).doit()
K.simplify()

Matrix([
[    d11*l_y*l_z/(9*l_x) + d22*l_x*l_z/(9*l_y) + d33*l_x*l_y/(9*l_z),  -d11*l_y*l_z/(9*l_x) + d22*l_x*l_z/(18*l_y) + d33*l_x*l_y/(18*l_z), -d11*l_y*l_z/(18*l_x) - d22*l_x*l_z/(18*l_y) + d33*l_x*l_y/(36*l_z),   d11*l_y*l_z/(18*l_x) - d22*l_x*l_z/(9*l_y) + d33*l_x*l_y/(18*l_z),   d11*l_y*l_z/(18*l_x) + d22*l_x*l_z/(18*l_y) - d33*l_x*l_y/(9*l_z), -d11*l_y*l_z/(18*l_x) + d22*l_x*l_z/(36*l_y) - d33*l_x*l_y/(18*l_z), -d11*l_y*l_z/(36*l_x) - d22*l_x*l_z/(36*l_y) - d33*l_x*l_y/(36*l_z),  d11*l_y*l_z/(36*l_x) - d22*l_x*l_z/(18*l_y) - d33*l_x*l_y/(18*l_z)],
[ -d11*l_y*l_z/(9*l_x) + d22*l_x*l_z/(18*l_y) + d33*l_x*l_y/(18*l_z),     d11*l_y*l_z/(9*l_x) + d22*l_x*l_z/(9*l_y) + d33*l_x*l_y/(9*l_z),   d11*l_y*l_z/(18*l_x) - d22*l_x*l_z/(9*l_y) + d33*l_x*l_y/(18*l_z), -d11*l_y*l_z/(18*l_x) - d22*l_x*l_z/(18*l_y) + d33*l_x*l_y/(36*l_z), -d11*l_y*l_z/(18*l_x) + d22*l_x*l_z/(36*l_y) - d33*l_x*l_y/(18*l_z),   d11*l_y*l_z/(18*l_x) + d22*l_x*l_z/(18*l_y) - d33*l_x*l_y/(9*l_z),  d11*l_y*l_z/(36*l_x) 

In [57]:
l = sp.symbols('l')
d = sp.symbols('d')
K_cube = K.subs({l_x: l, l_y: l, l_z: l, d11: d, d22: d, d33: d}).factor()
K_cube_coef = K_cube  * 12 / d / l
K_cube_coef

Matrix([
[ 4,  0, -1,  0,  0, -1, -1, -1],
[ 0,  4,  0, -1, -1,  0, -1, -1],
[-1,  0,  4,  0, -1, -1,  0, -1],
[ 0, -1,  0,  4, -1, -1, -1,  0],
[ 0, -1, -1, -1,  4,  0, -1,  0],
[-1,  0, -1, -1,  0,  4,  0, -1],
[-1, -1,  0, -1, -1,  0,  4,  0],
[-1, -1, -1,  0,  0, -1,  0,  4]])

In [58]:
# defining the mass matrix
rho = sp.symbols('rho')
M = sp.Integral(N * rho * N.T, (x, 0, l_x), (y, 0, l_y), (z, 0, l_z)).doit()
M_cube = M.subs({l_x: l, l_y: l, l_z: l}).factor()
M_cube_coef = M_cube / (l**3 * rho) * 216
M_cube_coef.simplify()


Matrix([
[8, 4, 2, 4, 4, 2, 1, 2],
[4, 8, 4, 2, 2, 4, 2, 1],
[2, 4, 8, 4, 1, 2, 4, 2],
[4, 2, 4, 8, 2, 1, 2, 4],
[4, 2, 1, 2, 8, 4, 2, 4],
[2, 4, 2, 1, 4, 8, 4, 2],
[1, 2, 4, 2, 2, 4, 8, 4],
[2, 1, 2, 4, 4, 2, 4, 8]])

In [60]:
lhs_cube = K_cube + M_cube
lhs_cube

Matrix([
[   d*l/3 + l**3*rho/27,            l**3*rho/54, -d*l/12 + l**3*rho/108,            l**3*rho/54,            l**3*rho/54, -d*l/12 + l**3*rho/108, -d*l/12 + l**3*rho/216, -d*l/12 + l**3*rho/108],
[           l**3*rho/54,    d*l/3 + l**3*rho/27,            l**3*rho/54, -d*l/12 + l**3*rho/108, -d*l/12 + l**3*rho/108,            l**3*rho/54, -d*l/12 + l**3*rho/108, -d*l/12 + l**3*rho/216],
[-d*l/12 + l**3*rho/108,            l**3*rho/54,    d*l/3 + l**3*rho/27,            l**3*rho/54, -d*l/12 + l**3*rho/216, -d*l/12 + l**3*rho/108,            l**3*rho/54, -d*l/12 + l**3*rho/108],
[           l**3*rho/54, -d*l/12 + l**3*rho/108,            l**3*rho/54,    d*l/3 + l**3*rho/27, -d*l/12 + l**3*rho/108, -d*l/12 + l**3*rho/216, -d*l/12 + l**3*rho/108,            l**3*rho/54],
[           l**3*rho/54, -d*l/12 + l**3*rho/108, -d*l/12 + l**3*rho/216, -d*l/12 + l**3*rho/108,    d*l/3 + l**3*rho/27,            l**3*rho/54, -d*l/12 + l**3*rho/108,            l**3*rho/54],
[-d*l/12 + l**3*rho/1

In [64]:
x_vector = sp.Matrix(n_element_nodes, 1, lambda i,_: sp.var("X_{i}".format(i=i)))
x_vector
y_vector = lhs_cube * x_vector

replacements, reduced_exprs = sp.cse(y_vector)

for var, expr in replacements:
    print(sp.ccode(expr,assign_to=repr(var)))


for i in range(8):
    print(f'rResult[equation_id_{i} + i]+={sp.ccode(reduced_exprs[0][i,0])};')

x0 = d*l;
x1 = pow(l, 3);
x2 = rho*x1;
x3 = (1.0/3.0)*x0 + (1.0/27.0)*x2;
x4 = (1.0/12.0)*x0;
x5 = (1.0/216.0)*rho*x1 - x4;
x6 = (1.0/108.0)*rho*x1 - x4;
x7 = X_5*x6;
x8 = X_7*x6;
x9 = (1.0/54.0)*x2;
x10 = X_1*x9;
x11 = X_3*x9;
x12 = x10 + x11 + x7 + x8;
x13 = X_2*x6 + X_4*x9;
x14 = X_4*x6;
x15 = X_6*x6;
x16 = X_0*x9;
x17 = X_2*x9;
x18 = x14 + x15 + x16 + x17;
x19 = X_3*x6 + X_5*x9;
x20 = X_0*x6 + X_6*x9;
x21 = X_1*x6 + X_7*x9;
x22 = x19 + x21;
x23 = x13 + x20;
rResult[equation_id_0 + i]+=X_0*x3 + X_6*x5 + x12 + x13;
rResult[equation_id_1 + i]+=X_1*x3 + X_7*x5 + x18 + x19;
rResult[equation_id_2 + i]+=X_2*x3 + X_4*x5 + x12 + x20;
rResult[equation_id_3 + i]+=X_3*x3 + X_5*x5 + x18 + x21;
rResult[equation_id_4 + i]+=X_2*x5 + X_4*x3 + x15 + x16 + x22;
rResult[equation_id_5 + i]+=X_3*x5 + X_5*x3 + x10 + x23 + x8;
rResult[equation_id_6 + i]+=X_0*x5 + X_6*x3 + x14 + x17 + x22;
rResult[equation_id_7 + i]+=X_1*x5 + X_7*x3 + x11 + x23 + x7;
